## Cálculo da emissão e distância percorrida 

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [1]:
import pandas as pd
import random
import numpy as np
import random
import geopandas as gpd
from utils.save_csv import save_csv
from utils.load_csv import load_csv
from datetime import datetime

In [2]:
consumo_diesel = load_csv('consumo_diesel.csv')

df_posicoes = load_csv('df_posicoes.csv')

In [3]:
#gdf = gpd.GeoDataFrame(df_posicoes, geometry=gpd.points_from_xy(df_posicoes.lon, df_posicoes.lat), crs="EPSG:4326")

#gdf = gdf.to_crs("EPSG:31983")

#gdf

In [4]:
#gdf["x"] = gdf.geometry.x
#gdf["y"] = gdf.geometry.y

#gdf.head()

In [5]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]

consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [6]:
def get_tipo_onibus(id_onibus):

    onibus = df_posicoes[df_posicoes['id_onibus']==id_onibus]

    return onibus['modelo'].unique()[0]

In [7]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2

In [8]:
df_posicoes['timestamp'] = pd.to_datetime(df_posicoes['timestamp'])

In [9]:
#gdf['id_onibus'].nunique()

In [10]:
data_frame = []

#for id_onibus in gdf['id_onibus'].unique():
    #df_onibus = gdf[gdf['id_onibus'] == id_onibus].reset_index(drop=True)
    #df_onibus.sort_values(by='timestamp', inplace=True)

for id_onibus in df_posicoes['id_onibus'].unique():
    df_onibus = (df_posicoes[df_posicoes['id_onibus'] == id_onibus].sort_values(by='timestamp').reset_index(drop=True))

    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['lon']
            x_depois = df_onibus.loc[i+1, 'lon']
            y_atual = row['lat']
            y_depois = df_onibus.loc[i+1, 'lat']
            momento_inicial = row['timestamp']
            momento_final = df_onibus.loc[i+1, 'timestamp']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_diesel = get_fator_consumo(tipo_veiculo)
            
            dist = np.sqrt((x_depois - x_atual)**2 + (y_depois - y_atual)**2)
            
            dados = {
                'id_onibus' : row['id_onibus'],
                'linha' : row['linha'],
                'is_eletrico' : row['is_eletrico'],
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_diesel_km' : litro_diesel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

gdf_final = pd.DataFrame(data_frame)
gdf_final.head()

,id_onibus,linha,is_eletrico,distancia_percorrida,ponto_inicial,ponto_final,momento_inicial,momento_final,modelo,litro_diesel_km,emissao_no2,emissao_co2
0,11897,917H-10,False,0.000099,"[-46.726082, -23.486366500000003]","[-46.726151, -23.486438]",2025-09-23 10:19:04-03:00,2025-09-23 10:19:48-03:00,Padron,0.63,0.000003,0.000005
1,11897,917H-10,False,0.000583,"[-46.726151, -23.486438]","[-46.7257505, -23.486014]",2025-09-23 10:19:48-03:00,2025-09-23 10:22:45-03:00,Padron,0.63,0.000016,0.000030
2,11897,917H-10,False,0.000000,"[-46.7257505, -23.486014]","[-46.7257505, -23.486014]",2025-09-23 10:22:45-03:00,2025-09-23 10:23:30-03:00,Padron,0.63,0.000000,0.000000
3,11897,917H-10,False,0.000000,"[-46.7257505, -23.486014]","[-46.7257505, -23.486014]",2025-09-23 10:23:30-03:00,2025-09-23 10:24:14-03:00,Padron,0.63,0.000000,0.000000
4,11897,917H-10,False,0.000484,"[-46.7257505, -23.486014]","[-46.726082, -23.486366500000003]",2025-09-23 10:24:14-03:00,2025-09-23 10:24:58-03:00,Padron,0.63,0.000013,0.000025


In [11]:
#gdf_final['momento_inicial'] = pd.to_datetime(gdf_final['momento_inicial'])

In [12]:
dados_trips_layer = []

for onibus in gdf_final['id_onibus'].unique():

    df_onibus = gdf_final[gdf_final['id_onibus'] == onibus].reset_index(drop=True)
    coordinates = []
    timestamps = []
    emissao_total = 0
    df_onibus.sort_values(by='momento_inicial', inplace=True)
    for i, row in df_onibus.iterrows():
        coordinates.append(row['ponto_inicial'])
        stamp = datetime.fromisoformat(row['momento_inicial'].isoformat())
        timestamps.append(stamp)
        emissao_total += row['emissao_co2']

    dados_trips_layer.append({
        'id_onibus': onibus,
        'linha': row['linha'],
        'emissao_co2': emissao_total,
        'is_eletrico': row['is_eletrico'],
        'coordinates' : coordinates,
        'timestamps' : timestamps
    })


df_trips = pd.DataFrame(dados_trips_layer)

In [13]:
def init_on_zero(timestamp_list)->list[float]:
    
    inicial = timestamp_list[0]

    new_timestamps = [ts - inicial for ts in timestamp_list]

    return [int(ts.total_seconds()) for ts in new_timestamps]

df_trips['timestamps'] = df_trips['timestamps'].apply(init_on_zero)

In [14]:
coord_t = str(df_trips.loc[0, 'coordinates'])

In [15]:
eval(coord_t)

[[-46.726082, -23.486366500000003],
 [-46.726151, -23.486438],
 [-46.7257505, -23.486014],
 [-46.7257505, -23.486014],
 [-46.7257505, -23.486014],
 [-46.726082, -23.486366500000003]]

In [16]:
df_trips.loc[0, 'timestamps']

[0, 44, 221, 266, 310, 354]

In [17]:
df_trips = df_trips[['coordinates', 'timestamps']]

In [18]:
save_csv(gdf_final, 'gdf_final.csv')

save_csv(df_trips, 'df_trips.csv')

Base salva em data\gdf_final.csv
Base salva em data\df_trips.csv
